In [2]:
%pip install tensorflow
%pip install pandas 
%pip install numpy

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.


In [3]:
%load_ext autoreload

In [4]:
%autoreload 2
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import datetime
import os
from feature_engineering.tensor_features import develop_features, convert_timestamp_to_seconds

from data_handler import LocalToLargeDataLoader

In [5]:
data_loader = LocalToLargeDataLoader(print_progress=True)
features, y_raw, test_features, y_raw_test = data_loader.load_raw_training_data() 

Retrieving training data...


In [6]:
feature_array, y = develop_features(features, y_raw)
test_feature_array, y_test = develop_features(test_features, y_raw_test)

floating_feature_array=np.array([np.array([convert_timestamp_to_seconds(x) for x in array]) for array in feature_array])
floating_test_feature_array=np.array([np.array([convert_timestamp_to_seconds(x) for x in array]) for array in test_feature_array])

In [7]:
x_train = tf.convert_to_tensor(floating_feature_array)
x_test = tf.convert_to_tensor(floating_test_feature_array)

In [8]:
y_train = np.array(y)
y_test = np.array(y_test)

input_shape = x_test[0].shape
print(input_shape)

(37,)


In [11]:
parameters=input_shape[0]
layer_n=1
layer=[
        tf.keras.layers.Dense(parameters//layer_n+1, activation='relu'),
        tf.keras.layers.Dropout(0.2)
        ]

model_list=[tf.keras.layers.Flatten(input_shape=input_shape)]
model_list.extend(layer*layer_n)
model_list.append(tf.keras.layers.Dense(2, activation='linear'))

model = tf.keras.models.Sequential(model_list)

model.compile(optimizer='adam',
  loss='mean_squared_error',
  metrics=['mae','mape'])

model.summary()

early_stopping = EarlyStopping(
    monitor='val_loss',  # You can change this to 'val_accuracy' or another metric
    patience=5,          # Number of epochs to wait for improvement
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity
)

checkpoint = ModelCheckpoint(
    filepath='modelepoch{epoch:02d}val_loss{val_loss:.2f}.keras',
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False,
    mode='min',
    verbose=1
)

log_dir= os.path.join('logs','fit',datetime.datetime.now().strftime("%Y%m%d-%H%M%S"),)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)             │ (None, 37)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 38)             │         1,444 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 38)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │            78 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,522 (5.95 KB)

 Trainable params: 1,522 (5.95 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
print(f"Y data samples {y_train[:5]}")
model.fit(x_train, y_train, epochs=20, batch_size=32, validation_split=0.2, callbacks=[checkpoint, early_stopping, tensorboard_callback])

Y data samples [[  36.84686    5.84638]
 [  48.5332    -6.12003]
 [  28.82071 -120.46513]
 [  33.86918 -118.81433]
 [  33.18068   -9.43055]]
Epoch 1/20
 1/14 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - loss: 333143433281536.0000 - mae: 15137526.0000 - mape: 118499152.0000

FailedPreconditionError: {{function_node __wrapped__CreateSummaryFileWriter_device_/job:localhost/replica:0/task:0/device:CPU:0}} logs\fit is not a directory [Op:CreateSummaryFileWriter] name: 